In [1]:
import os
from openai import OpenAI
from typing_extensions import override
import pandas as pd
from openai import AssistantEventHandler
from IPython.core.display import display, HTML
import webbrowser
import tempfile
import re
import warnings
warnings.filterwarnings('ignore')

client = OpenAI()

/var/folders/7v/6lrll0c16r142j8200yccjz00000gn/T/ipykernel_53327/1793699707.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [8]:
instructions = """
You are an assistant that points users to which individuals have similar research interests using the documents provided to you.
When a user states their interests, you must provide a summary of the individuals that most resemble the user's interests and 
explain what in particular they have done or are doing that is similar to the user's interests.
Please use newlines to separate every single one of your sentences.
"""

query = """I'm interested in solar panels for EVs."""

In [9]:
# Step 1: Create a new Assistant with File Search Enabled
assistant = client.beta.assistants.create(
    name="Research Expertise Assistant",
    instructions=instructions,
    model="gpt-4o",
    tools=[{"type": "file_search"}],
)

# Step 2: Upload files and add them to a Vector Store
vector_store = client.beta.vector_stores.create(name="PI Bios")
file_paths = ["data/file1.txt"]
file_streams = [open(path, "rb") for path in file_paths]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# Step 3: Update the assistant to use the new Vector Store
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# Step 4: Query the assistant with question
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)
    
    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)
    
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)
    
    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

thread = client.beta.threads.create(
        messages=[{"role": "user", "content": query, "attachments": []}],
        tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
    )

In [10]:
# Step 5: Stream the answer
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > file_search


assistant > Here are some researchers whose work aligns with your interest in solar panels for electric vehicles (EVs):

1. **Rajeevan Amirtharajah**:
   - **Research Interests**: Focuses on the development and implementation of electronic systems powered by energy harvested from the environment. His research includes the conversion of incident light (solar energy harvesting) into electrical form and power electronics to generate stable supply voltages. He works on integrated solar cells for low-power applications, among other energy-harvesting technologies.
   - **Applicable Work**: Rajeevan’s research on integrated solar cells and energy harvesting from light could directly contribute to innovations in solar panels for EVs.
   - **Affiliation**: University of California, Davis, Department of Electrical and Computer Engineering.
   - **More Information**: [Rajeevan Amirtharajah's Profile](https://faculty.engineering.ucdavis.edu/amirtharajah/biography/)【4:0†s